[View in Colaboratory](https://colab.research.google.com/github/simonoppowa/FashionMNIST-Deep-Feedforward-Neural-Network/blob/master/FashionMNIST_Assignment1.ipynb)

# Fashion MNIST Deep Feedforward Neural Network
## Assignment 1
## Simon Oppowa

[Fashion MNIST Dataset By Zalando](https://github.com/zalandoresearch/fashion-mnist)

### Test Accuracy:



*   `Accuracy after 10 epchos: 0.8730000257492065`
*   `Accuracy after 50 epchos: 0.8912000060081482`
*   `Accuracy after 100 epchos: 0.8959000110626221`
*   `Accuracy after 150 epchos: 0.8991000056266785`



In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import math
from tensorflow.examples.tutorials.mnist import input_data

## Download/Extract Fashion MNIST Dataset

In [47]:
mnist = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/', one_hot=True)

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


## Define Network Constants


*    NUM_EPOCHS: Number of times the training data gets iterated through
*   MINIBATCH_SIZE: Size of one data to train at a time
*   PKEEP: Dropout Rate for overfitting problem (Probability for a neuron to be dropped at one iteration)
* LEARNING_RATE: Learning rate at start



In [0]:
NUM_EPOCHS = 150
MINIBATCH_SIZE = 1000
PKEEP = 0.75
LEARNING_RATE = 0.003

## Define Network Size



1.   Input Layer (784 Neurons: 28x28 pixels as input)
2.   First HIdden Layer (200 Neurons)
3.   Second Hidden Layer (100 Neurons)
4.   Third Hidded Layer (60 Neurons)
5.   Fourth Hidden Layer (30 Neurons)
6.   Output Layer (10 Neurons: )




In [0]:
FIRST_LEVEL_NEURONS = 784
SECOND_LEVEL_NEURONS = 200
THIRD_LEVEL_NEURONS = 100
FOURTH_LEVEL_NEURONS = 60
FIFTH_LEVEL_NEURONS = 30
SIXTH_LEVEL_NEURONS = 10

## Define Tensorflow Placeholders

*   X: Input Placeholder
*   Y_: Output Placeholder with real values



*   step: current step for calculation of learning rate
*   pkeep: placeholder for dropout rate


In [0]:
X = tf.placeholder(tf.float32, [None, 784])
Y_ = tf.placeholder(tf.float32, [None, 10])

step = tf.placeholder(tf.int32)
pkeep = tf.placeholder(tf.float32)

## Define Weights and Biases


*   Weight are initialized with small random values between -0.2 and +0.2
*   Biases are initialized with small positive values to work best with RELU



In [0]:
W1 = tf.Variable(tf.truncated_normal([FIRST_LEVEL_NEURONS, SECOND_LEVEL_NEURONS], stddev=0.1))
b1 = tf.Variable(tf.ones([SECOND_LEVEL_NEURONS])/10)

W2 = tf.Variable(tf.truncated_normal([SECOND_LEVEL_NEURONS, THIRD_LEVEL_NEURONS], stddev=0.1))
b2 = tf.Variable(tf.ones([THIRD_LEVEL_NEURONS])/10)

W3 = tf.Variable(tf.truncated_normal([THIRD_LEVEL_NEURONS, FOURTH_LEVEL_NEURONS], stddev=0.1))
b3 = tf.Variable(tf.ones([FOURTH_LEVEL_NEURONS])/10)

W4 = tf.Variable(tf.truncated_normal([FOURTH_LEVEL_NEURONS, FIFTH_LEVEL_NEURONS], stddev=0.1))
b4 = tf.Variable(tf.ones([FIFTH_LEVEL_NEURONS])/10)

W5 = tf.Variable(tf.truncated_normal([FIFTH_LEVEL_NEURONS, SIXTH_LEVEL_NEURONS], stddev=0.1))
b5 = tf.Variable(tf.ones([SIXTH_LEVEL_NEURONS])/10)

## The Model

`LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX`

`Drop neurons after every step with pkeep rate`
### Forward propagation

In [0]:
Y1 = tf.nn.relu(tf.matmul(X, W1) + b1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + b2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + b3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + b4)
Y4d = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(Y4d, W5) + b5
Y = tf.nn.softmax(Ylogits)

### Cross Entropy Loss Function

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*MINIBATCH_SIZE # Normalize cross entropy for MINIBATCH_SIZE images

Calculate accuracy of trained model

In [0]:
is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Start with a LEARNING_RATE learning rate and decrease as you get closer to minimum

In [0]:
lr = 0.0001 +  tf.train.exponential_decay(LEARNING_RATE, step, 5000, 1/math.e)
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

## Run Training algorithm


In [56]:
# Create Tensorflow session
with tf.Session() as sess:
  
    #Initialize variables
    sess.run(tf.global_variables_initializer())

    # Training loop
    for epoch in range(NUM_EPOCHS):
      
      # Calculate minibatches_size
        m = len(mnist.train.images)
        num_minibatches = int(m / MINIBATCH_SIZE)

        for i in range(num_minibatches):
          
            # load batch of images and correct answers
            batch_X, batch_Y = mnist.train.next_batch(MINIBATCH_SIZE)
            train_data = {X: batch_X, Y_: batch_Y, step: i, pkeep: PKEEP}

            # train (backpropagation)
            sess.run(train_step, feed_dict=train_data)

            a, c = sess.run([accuracy, cross_entropy], feed_dict=train_data)

        # Print accuracy of train and test images after every epoch
        print("Accuracy after epoch {epoch_num}:".format(epoch_num=epoch))
        print("     Train accuracy: {accuracy}".format(accuracy=a))
        
        # Load test data and test
        test_data = {X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1.0}
        d, e = sess.run([accuracy, cross_entropy], feed_dict=test_data)
        print("     Test accuracy: {accuracy}".format(accuracy=d))



Accuracy after epoch 0:
     Train accuracy: 0.7390000224113464
     Test accuracy: 0.7832000255584717
Accuracy after epoch 1:
     Train accuracy: 0.8159999847412109
     Test accuracy: 0.8264999985694885
Accuracy after epoch 2:
     Train accuracy: 0.8360000252723694
     Test accuracy: 0.8482000231742859
Accuracy after epoch 3:
     Train accuracy: 0.8510000109672546
     Test accuracy: 0.8539999723434448
Accuracy after epoch 4:
     Train accuracy: 0.8669999837875366
     Test accuracy: 0.8575999736785889
Accuracy after epoch 5:
     Train accuracy: 0.8629999756813049
     Test accuracy: 0.8578000068664551
Accuracy after epoch 6:
     Train accuracy: 0.8700000047683716
     Test accuracy: 0.867900013923645
Accuracy after epoch 7:
     Train accuracy: 0.8679999709129333
     Test accuracy: 0.8655999898910522
Accuracy after epoch 8:
     Train accuracy: 0.8740000128746033
     Test accuracy: 0.8720999956130981
Accuracy after epoch 9:
     Train accuracy: 0.8790000081062317
     Test 